# Projet : Scrapping du site Boulanger

Je vais scrapper le site boulanger pour récupérer les informations des ordinateurs portables i7  
Les données que je vais récupérer sont  : 
- Titre
- Note
- Prix
- Moniteur_Taille de l'écran
- Moniteur_Equivalence
- Moniteur_Résolution de l'écran
- Logiciels_Système d'exploitation
- Logiciels_Version
- Processeur_Référence et spécificités
- Mémoire vive_Capacité totale
- Mémoire vive_Type
- Stockage_Capacité du disque dur
- Stockage_Port du disque dur
- Stockage_Vitesse de rotation
- Stockage_Capacité du SSD
- Stockage_Port du SSD
- Dimensions_Dimensions l x h x p
- Dimensions_Poids
- Carte vidéo_Carte

Importation des librairies

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import numpy as np
import pandas as pd
import time

On ouvre la page web

In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.boulanger.com/')

Définition d'une fonction qui va produire un delai après chaque click, pour simuler une activité humaine

In [ ]:
def generate_number_delay():
    """
    Retourne un chiffre aléatoire suivant une distribution normal de moyenne 4 secondes et d'ecart-type de 0.8
    Ce chiffre aléatoire est le délai après chaque clic. Le but est de simuler le comportement d'un humain :
    un délai fixe peut attirer l'attention des contrôleurs, tout comme un délai aléatoire d'une distribution uniforme
    """
    mean = 4
    sigma = 0.8
    delai = np.random.normal(mean,sigma,1)[0]
    while delai < 0:
        delai = np.random.normal(mean,sigma,1)[0]
    time.sleep(delai)

On aggrandi la fenêtre du navigateur

In [ ]:
browser.maximize_window()

On cherche l'input de la barre de recherche et on envoie le text à mettre dans l'input

In [ ]:
search_input = browser.find_element_by_xpath('//*[@id="o-header_search_id"]')
search_input.send_keys("ordinateur portable")

On cherche le bouton recherche pour pouvoir cliquer dessus et envoyer notre requête.  
On appelle la fonction generate_number_delay() pour mettre un temps t'attente après le click

In [ ]:
search_button = browser.find_element_by_xpath('//*[@id="container"]/header/section[2]/div[1]/section/form/p/button')
search_button.click()
generate_number_delay()

On cherche a filtrer la recherche en cliquant sur le processeur Intel Core i7

In [ ]:
filtre_i7 = browser.find_element_by_xpath('//*[@id="multi_facet_points_comparateurs_____processeur_3"]/span[1]')
filtre_i7.click()
generate_number_delay()

On stocke dans liste_desc_pc la liste des pc présent sur la page (30 par page)
et on affiche le titre de chaque article pour vérifier que ça récupère ce qu'on veut

In [ ]:
liste_desc_pc = browser.find_elements_by_css_selector('div.designations h2 a')
[titre.text for titre in liste_desc_pc]

On créé une liste de caractéristique qu'on souhaite récupérer, puis on initialise un dictionnaire qui va contenir pour chaque caractéristique une liste de données

In [ ]:
characteristics_list = ["Moniteur_Taille de l'écran", "Moniteur_Equivalence", "Moniteur_Résolution de l'écran", "Logiciels_Système d'exploitation", "Logiciels_Version", "Processeur_Référence et spécificités", "Mémoire vive_Capacité totale", "Mémoire vive_Type", "Stockage_Capacité du disque dur", "Stockage_Port du disque dur", "Stockage_Vitesse de rotation", "Stockage_Capacité du SSD", "Stockage_Port du SSD", "Dimensions_Dimensions l x h x p", "Dimensions_Poids", "Carte vidéo_Carte"]
dico = {}
dico["Titre"] = []
dico["Note"] = []
dico["Prix"] = []
for characteristic in characteristics_list:
    dico[characteristic] = []
dico

Dans next_page_class_list il y a une list de nom de class associé au bouton page suivante, lorsque cette liste contient la class "navPage-inactive", cela veut dire qu'on est arrivé à la dernière page.  
Pour chaque page, on récupère la liste d'article, puis pour chaque article on recupère d'abord le titre qu'on va ajouter aux dictionnaire, puis on click sur l'article et on récupère les autres caractéristiques.  
A chaque fois qu'on récupère un caractéristique on le supprime de la liste des caractéristiques (c_list) qu'on a copié au préalable, cela permet de gérer les données manquantes.  
Après avoir ajouté toutes les caractéristiques dans le dictionnaire, si il reste des caractéristiques dans c_list cela veut dire qu'il y a des données manquantes, on les ajoutes donc dans le dictionnaire en tant que NaN.

In [ ]:
next_page_class_list = browser.find_element_by_class_name("navPage-right").get_attribute("class").split()
while "navPage-inactive" not in next_page_class_list:
    liste_desc_pc = browser.find_elements_by_css_selector('div.designations h2 a')
    for i in range(len(liste_desc_pc)):
        title = liste_desc_pc[i].text
        dico["Titre"].append(title)
        liste_desc_pc[i].click()
        generate_number_delay()
        try:
            rating = int(browser.find_element_by_class_name("rating").get_attribute("class").split()[2].split('_')[1]) * 0.1
        except:
            rating = np.nan
        dico["Note"].append(rating)
        price = float(browser.find_element_by_class_name("fix-price").text.split()[0].replace('€', '.'))
        dico["Prix"].append(price)
        more = browser.find_element_by_class_name("more")
        more.click()
        generate_number_delay()
        characteristics = browser.find_element_by_class_name('characteristic').text
        feature = ""
        c_list = characteristics_list.copy()
        for characteristic in characteristics.split("\n"):
            characteristic = re.sub(' \(.+\)', '', characteristic)
            if ":" not in characteristic:
                feature = characteristic
            else :
                key = (feature + "_" + characteristic).split(":")[0].strip()
                value = (feature + "_" + characteristic).split(":")[1].strip()
                if (key in c_list):
                    dico[key].append(value)
                    c_list.remove(key)
        for c in c_list:
            dico[c].append(np.nan)
        browser.back()
        liste_desc_pc = browser.find_elements_by_css_selector('div.designations h2 a')
    next_page = browser.find_element_by_class_name("navPage-right")
    next_page_class_list = next_page.get_attribute("class").split()
    next_page.click()
    generate_number_delay()

In [ ]:
dico

On transforme notre dictionnaire en dataframe

In [ ]:
df = pd.DataFrame(dico)
df

On renomme les colonnes pour enlever les espaces, les accents et pour ajouter les unités

In [ ]:
df_rename = df.rename(index=str, columns={"Moniteur_Taille de l'écran" : "Moniteur_Taille_ecran_(pouces)", \
                                "Moniteur_Equivalence" : "Moniteur_Equivalence_(cm)", \
                                "Moniteur_Résolution de l'écran" : "Moniteur_Resolution_ecran_(pixels)", \
                                "Logiciels_Système d'exploitation" : "Logiciels_Systeme_exploitation", \
                                "Logiciels_Version" : "Logiciels_Version_(Bits)", \
                                "Processeur_Référence et spécificités" : "Processeur_Reference", \
                                "Mémoire vive_Capacité totale" : "Memoire_vive_Capacite_totale_(Go)", \
                                "Mémoire vive_Type" : "Memoire_vive_Type", \
                                "Stockage_Capacité du disque dur" : "Stockage_Capacite_disque_dur_(Go)", \
                                "Stockage_Port du disque dur" : "Stockage_Port_disque_dur", \
                                "Stockage_Vitesse de rotation" : "Stockage_Vitesse_rotation(tours/min)", \
                                "Stockage_Capacité du SSD" : "Stockage_Capacite_SSD_(Go)", \
                                "Stockage_Port du SSD" : "Stockage_Port_SSD", \
                                "Dimensions_Dimensions l x h x p" : "Dimensions_l_x_h_x_p_(cm)", \
                                "Dimensions_Poids" : "Dimensions_Poids_(g)", \
                                "Carte vidéo_Carte" : "Carte_video"})
df_rename.head(1)

On transforme les données en numerique et on enlève les unités

In [ ]:
def clean_data(row):
    row_cleaned = row
    if str(row["Moniteur_Taille_ecran_(pouces)"]) != "nan":
        row_cleaned["Moniteur_Taille_ecran_(pouces)"] = float(row["Moniteur_Taille_ecran_(pouces)"].replace(" pouces", "").replace(",", "."))
    if str(row["Moniteur_Equivalence_(cm)"]) != "nan":
        row_cleaned["Moniteur_Equivalence_(cm)"] = float(row["Moniteur_Equivalence_(cm)"].replace(" cm", "").replace(",", "."))
    if str(row["Moniteur_Resolution_ecran_(pixels)"]) != "nan":
        row_cleaned["Moniteur_Resolution_ecran_(pixels)"] = row["Moniteur_Resolution_ecran_(pixels)"].replace(" pixels", "")
    if str(row["Logiciels_Version_(Bits)"]) != "nan":
        row_cleaned["Logiciels_Version_(Bits)"] = int(row["Logiciels_Version_(Bits)"].replace(" Bits", ""))
    if str(row["Memoire_vive_Capacite_totale_(Go)"]) != "nan":
        row_cleaned["Memoire_vive_Capacite_totale_(Go)"] = int(row["Memoire_vive_Capacite_totale_(Go)"].replace(" Go", ""))
    if str(row["Stockage_Capacite_disque_dur_(Go)"]) != "nan" and "To" in row["Stockage_Capacite_disque_dur_(Go)"]:
        row_cleaned["Stockage_Capacite_disque_dur_(Go)"] = float(row["Stockage_Capacite_disque_dur_(Go)"].replace(" To", "").replace(",", ".")) * 1000
    elif str(row["Stockage_Capacite_disque_dur_(Go)"]) != "nan":
        row_cleaned["Stockage_Capacite_disque_dur_(Go)"] = float(row["Stockage_Capacite_disque_dur_(Go)"].replace(" Go", "").replace(",", "."))
    if str(row["Stockage_Vitesse_rotation(tours/min)"]) not in ["nan", "Non communiqué"]:
        row_cleaned["Stockage_Vitesse_rotation(tours/min)"] = int(row["Stockage_Vitesse_rotation(tours/min)"].replace(" tours/min", ""))
    elif str(row["Stockage_Vitesse_rotation(tours/min)"]) == "Non communiqué":
        row_cleaned["Stockage_Vitesse_rotation(tours/min)"] = np.nan
    if str(row["Stockage_Capacite_SSD_(Go)"]) != "nan" and "To" in row["Stockage_Capacite_SSD_(Go)"]:
        row_cleaned["Stockage_Capacite_SSD_(Go)"] = float(row["Stockage_Capacite_SSD_(Go)"].replace(" To", "").replace(",", ".")) * 1000
    elif str(row["Stockage_Capacite_SSD_(Go)"]) != "nan":
        row_cleaned["Stockage_Capacite_SSD_(Go)"] = float(row["Stockage_Capacite_SSD_(Go)"].replace(" Go", "").replace(",", "."))  
    if str(row["Dimensions_l_x_h_x_p_(cm)"]) != "nan":
        row_cleaned["Dimensions_l_x_h_x_p_(cm)"] = row["Dimensions_l_x_h_x_p_(cm)"].replace(" cm", "")
    if "kg" in row["Dimensions_Poids_(g)"]:
        row_cleaned["Dimensions_Poids_(g)"] = float(row["Dimensions_Poids_(g)"].replace(" kg", "").replace(",", ".")) * 1000
    else:
        row_cleaned["Dimensions_Poids_(g)"] = float(row["Dimensions_Poids_(g)"].replace(" g", "").replace(",", "."))
    return row_cleaned

df_cleaned = df_rename.apply(clean_data, axis=1)
df_cleaned

On converti le dataframe en fichier csv

In [ ]:
df_cleaned.to_csv(path_or_buf = "boulanger_scrapping.csv", index=False)

On peut vérifier qu'on a bien enregistré le fichier csv en l'ouvrant

In [ ]:
test = pd.read_csv("boulanger_scrapping.csv")
test

le code ci dessous réunit tout dans une fonction, il y a quatres paramètres:
- nb_page : le nombre de page a scrapper
- file_name : le nom du fichier csv
- page_num : la page où on commence à scrapper (utile si on veut reprendre après un plantage du code)
- article_num : le numéro de l'article où on commence à scrapper (utile si on veut reprendre après un plantage du code)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import re
import numpy as np
import pandas as pd
import time

def generate_number_delay():
    """
    Retourne un chiffre aléatoire suivant une distribution normal de moyenne 4 secondes et d'ecart-type de 0.8
    Ce chiffre aléatoire est le délai après chaque clic. Le but est de simuler le comportement d'un humain :
    un délai fixe peut attirer l'attention des contrôleurs, tout comme un délai aléatoire d'une distribution uniforme
    """
    mean = 4
    sigma = 0.8
    delai = np.random.normal(mean,sigma,1)[0]
    while delai < 0:
        delai = np.random.normal(mean,sigma,1)[0]
    time.sleep(delai)

def clean_data(row):
    row_cleaned = row
    if str(row["Moniteur_Taille_ecran_(pouces)"]) != "nan":
        value = str(row["Moniteur_Taille_ecran_(pouces)"]).replace(" pouces", "").replace(",", ".")
        row_cleaned["Moniteur_Taille_ecran_(pouces)"] = float(value)
    if str(row["Moniteur_Equivalence_(cm)"]) != "nan":
        value = str(row["Moniteur_Equivalence_(cm)"]).replace(" cm", "").replace(",", ".")
        row_cleaned["Moniteur_Equivalence_(cm)"] = float(value)
    if str(row["Moniteur_Resolution_ecran_(pixels)"]) != "nan":
        row_cleaned["Moniteur_Resolution_ecran_(pixels)"] = row["Moniteur_Resolution_ecran_(pixels)"].replace(" pixels", "")
    if str(row["Logiciels_Version_(Bits)"]) != "nan":
        value = str(row["Logiciels_Version_(Bits)"]).replace(" Bits", "")
        row_cleaned["Logiciels_Version_(Bits)"] = int(value)
    if str(row["Memoire_vive_Capacite_totale_(Go)"]) != "nan":
        value = str(row["Memoire_vive_Capacite_totale_(Go)"]).replace(" Go", "")
        row_cleaned["Memoire_vive_Capacite_totale_(Go)"] = int(value)
    if str(row["Stockage_Capacite_disque_dur_(Go)"]) != "nan" and "To" in str(row["Stockage_Capacite_disque_dur_(Go)"]):
        value = str(row["Stockage_Capacite_disque_dur_(Go)"]).replace(" To", "").replace(",", ".")
        row_cleaned["Stockage_Capacite_disque_dur_(Go)"] = float(value) * 1000
    elif str(row["Stockage_Capacite_disque_dur_(Go)"]) != "nan":
        value = str(row["Stockage_Capacite_disque_dur_(Go)"]).replace(" Go", "").replace(",", ".")
        row_cleaned["Stockage_Capacite_disque_dur_(Go)"] = float(value)
    if str(row["Stockage_Vitesse_rotation(tours/min)"]) not in ["nan", "Non communiqué"]:
        value = str(row["Stockage_Vitesse_rotation(tours/min)"]).replace(" tours/min", "")
        row_cleaned["Stockage_Vitesse_rotation(tours/min)"] = int(value)
    elif str(row["Stockage_Vitesse_rotation(tours/min)"]) == "Non communiqué":
        row_cleaned["Stockage_Vitesse_rotation(tours/min)"] = np.nan
    if str(row["Stockage_Capacite_SSD_(Go)"]) != "nan" and "To" in str(row["Stockage_Capacite_SSD_(Go)"]):
        value = str(row["Stockage_Capacite_SSD_(Go)"]).replace(" To", "").replace(",", ".")
        row_cleaned["Stockage_Capacite_SSD_(Go)"] = float(value) * 1000
    elif str(row["Stockage_Capacite_SSD_(Go)"]) != "nan":
        value = str(row["Stockage_Capacite_SSD_(Go)"]).replace(" Go", "").replace(",", ".")  
        row_cleaned["Stockage_Capacite_SSD_(Go)"] = float(value)  
    if str(row["Dimensions_l_x_h_x_p_(cm)"]) != "nan":
        row_cleaned["Dimensions_l_x_h_x_p_(cm)"] = row["Dimensions_l_x_h_x_p_(cm)"].replace(" cm", "")
    if "kg" in str(row["Dimensions_Poids_(g)"]):
        value = str(row["Dimensions_Poids_(g)"]).replace(" kg", "").replace(",", ".")
        row_cleaned["Dimensions_Poids_(g)"] = float(value) * 1000
    else:
        value = str(row["Dimensions_Poids_(g)"]).replace(" g", "").replace(",", ".")
        row_cleaned["Dimensions_Poids_(g)"] = float(value)
    return row_cleaned

def scrap_data(nb_page = 10, file_name = "boulanger_scrapping.csv", page_num = 1, article_num = 0):
    
    browser = webdriver.Chrome()
    if page_num == 1:
        browser.get('https://www.boulanger.com/')
        browser.maximize_window()
        
        search_input = browser.find_element_by_xpath('//*[@id="o-header_search_id"]')
        search_input.send_keys("ordinateur portable")
        search_button = browser.find_element_by_xpath('//*[@id="container"]/header/section[2]/div[1]/section/form/p/button')
        search_button.click()
        generate_number_delay()
        filtre_i7 = browser.find_element_by_xpath('//*[@id="multi_facet_points_comparateurs_____processeur_3"]/span[1]')
        filtre_i7.click()
        generate_number_delay()
    else:
        browser.get('https://www.boulanger.com/c/tous-les-ordinateurs-portables/points_comparateurs_____processeur~intel20core20i7?numPage={}'.format(page_num))
        browser.maximize_window()
    
    characteristics_list = ["Moniteur_Taille de l'écran", "Moniteur_Equivalence", "Moniteur_Résolution de l'écran", "Logiciels_Système d'exploitation", "Logiciels_Version", "Processeur_Référence et spécificités", "Mémoire vive_Capacité totale", "Mémoire vive_Type", "Stockage_Capacité du disque dur", "Stockage_Port du disque dur", "Stockage_Vitesse de rotation", "Stockage_Capacité du SSD", "Stockage_Port du SSD", "Dimensions_Dimensions l x h x p", "Dimensions_Poids", "Carte vidéo_Carte"]
    dico = {}
    dico["Titre"] = []
    dico["Note"] = []
    dico["Prix"] = []
    for characteristic in characteristics_list:
        dico[characteristic] = []
    page = 0
    next_page_class_list = browser.find_element_by_class_name("navPage-right").get_attribute("class").split()
    while page < nb_page and "navPage-inactive" not in next_page_class_list:
        liste_desc_pc = browser.find_elements_by_css_selector('div.designations h2 a')
        if article_num != 0:
            i = article_num
            browser.execute_script("arguments[0].scrollIntoView();", liste_desc_pc[i])
            article_num = 0
        else:
            i = 0
        while i < len(liste_desc_pc):
            title = liste_desc_pc[i].text
            dico["Titre"].append(title)
            liste_desc_pc[i].click()
            generate_number_delay()
            try:
                rating = int(browser.find_element_by_class_name("rating").get_attribute("class").split()[2].split('_')[1]) * 0.1
            except:
                rating = np.nan
            dico["Note"].append(rating)
            price = float(browser.find_element_by_class_name("fix-price").text.split()[0].replace('€', '.'))
            dico["Prix"].append(price)
            more = browser.find_element_by_class_name("more")
            more.click()
            generate_number_delay()
            characteristics = browser.find_element_by_class_name('characteristic').text
            feature = ""
            c_list = characteristics_list.copy()
            for characteristic in characteristics.split("\n"):
                characteristic = re.sub(' \(.+\)', '', characteristic)
                if ":" not in characteristic:
                    feature = characteristic
                else :
                    key = (feature + "_" + characteristic).split(":")[0].strip()
                    value = (feature + "_" + characteristic).split(":")[1].strip()
                    if (key in c_list):
                        dico[key].append(value)
                        c_list.remove(key)
            for c in c_list:
                dico[c].append(np.nan)
                
            browser.back()

            df = pd.DataFrame(dico)
            df_rename = df.rename(index=str, columns={"Moniteur_Taille de l'écran" : "Moniteur_Taille_ecran_(pouces)", \
                                        "Moniteur_Equivalence" : "Moniteur_Equivalence_(cm)", \
                                        "Moniteur_Résolution de l'écran" : "Moniteur_Resolution_ecran_(pixels)", \
                                        "Logiciels_Système d'exploitation" : "Logiciels_Systeme_exploitation", \
                                        "Logiciels_Version" : "Logiciels_Version_(Bits)", \
                                        "Processeur_Référence et spécificités" : "Processeur_Reference", \
                                        "Mémoire vive_Capacité totale" : "Memoire_vive_Capacite_totale_(Go)", \
                                        "Mémoire vive_Type" : "Memoire_vive_Type", \
                                        "Stockage_Capacité du disque dur" : "Stockage_Capacite_disque_dur_(Go)", \
                                        "Stockage_Port du disque dur" : "Stockage_Port_disque_dur", \
                                        "Stockage_Vitesse de rotation" : "Stockage_Vitesse_rotation(tours/min)", \
                                        "Stockage_Capacité du SSD" : "Stockage_Capacite_SSD_(Go)", \
                                        "Stockage_Port du SSD" : "Stockage_Port_SSD", \
                                        "Dimensions_Dimensions l x h x p" : "Dimensions_l_x_h_x_p_(cm)", \
                                        "Dimensions_Poids" : "Dimensions_Poids_(g)", \
                                        "Carte vidéo_Carte" : "Carte_video"})
            df_cleaned = df_rename.apply(clean_data, axis=1)
            df_cleaned.to_csv(path_or_buf = file_name, index=False)
            liste_desc_pc = browser.find_elements_by_css_selector('div.designations h2 a')
            i += 1
        page += 1
        next_page = browser.find_element_by_class_name("navPage-right")
        next_page_class_list = next_page.get_attribute("class").split()
        next_page.click()
        generate_number_delay()
    browser.close()

scrap_data()
# scrap_data(file_name = "boulanger_scrapping_after_timeout2.csv", page_num = 4, article_num = 12)

In [ ]:
data = pd.read_csv("boulanger_scrapping.csv")
data